# Prompt Versioning Tracker
Log each prompt iteration with parameters, outcome, and notes. Save locally as **prompt_versions.csv** or push to Google Sheets.

In [ ]:
!pip -q install pandas gspread oauth2client

In [ ]:
import pandas as pd, datetime, json
from pathlib import Path

LOG_PATH = Path("prompt_versions.csv")

# Load existing log or create empty DataFrame
if LOG_PATH.exists():
    df = pd.read_csv(LOG_PATH)
else:
    df = pd.DataFrame(columns=[
        "timestamp", "prompt_name", "version",
        "prompt_text", "parameters", "outcome", "notes"
    ])

def log_prompt(prompt_name: str,
               version: str,
               prompt_text: str,
               parameters: dict,
               outcome: str,
               notes: str = "") -> None:
    """
    Append a prompt run to the log.
    outcome: e.g., 'success', 'failure', 'bug', 'needs‑review'
    """
    global df
    entry = {
        "timestamp": datetime.datetime.utcnow().isoformat(timespec="seconds"),
        "prompt_name": prompt_name,
        "version": version,
        "prompt_text": prompt_text,
        "parameters": json.dumps(parameters),
        "outcome": outcome,
        "notes": notes
    }
    df = pd.concat([df, pd.DataFrame([entry])], ignore_index=True)
    df.to_csv(LOG_PATH, index=False)
    print(f"Logged {prompt_name} v{version} → {outcome}")

print("▶️ Use log_prompt(name, version, text, params_dict, outcome, notes)")
print("Example:")
example = {
    "name": "email_summarizer",
    "version": "0.1",
    "text": "Summarize the following email ...",
    "params": {"temperature": 0.3},
    "outcome": "success",
    "notes": "Initial baseline"
}
print(json.dumps(example, indent=2))

display(df.head())

### Google Sheets Sync (optional)
```python
import gspread
from oauth2client.service_account import ServiceAccountCredentials

creds_json = 'path/to/credentials.json'
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(creds_json, scope)
client = gspread.authorize(creds)
sheet = client.open('PromptVersions').sheet1
sheet.append_row(list(df.iloc[-1]))
```